In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import time
import serology
import distributions.priors as priors
import pickle
import emcee
import pandas as pd
%matplotlib inline

In [2]:
# Import the serology model (simulator)
Model = serology.Serology()

In [3]:
### SET UP UNIFORM PRIORS OVER CORE SEROLOGY PARAMETERS ###

# Lower limits
theta_lower = np.array([  0, # fzero
                          0, # gzero
                          5, # azero
                          np.log(0.05), # logHetBoosting
                          np.log(10), # logBaselineBoostingFactor
                          np.log(10), # logBaselineBoostingThreshold
                          np.log(0.05), # logAttenuationFactor
                          2,# NAdultAntibodies
                          np.log(0.01), # logRhoChild
                          np.log(0.01) # logRhoAdultExtra
                          ])

# Upper limits
theta_upper = np.array([  1.0, # fzero
                          1.0, # gzero
                          35, # azero
                          np.log(4), # logHetBoosting
                          np.log(30000), # logBaselineBoostingFactor
                          np.log(10000), # logBaselineBoostingThreshold
                          np.log(4), # logAttenuationFactor
                          20,# NAdultAntibodies
                          np.log(10), # logRhoChild
                          np.log(20) # logRhoAdultExtra
                          ])

theta_prior = priors.Uniform(lower=theta_lower, upper=theta_upper)

log_eir_prior = priors.Uniform(lower=np.log(5e-3), upper=np.log(50))

In [5]:
# Run simulations
nsims = int(1e7)
xs = np.zeros((nsims, 1))
ps = np.zeros((nsims, 10 + 1 + 1)) # n_parameters = 10 (serology) + 1 (log_eir) + 1 (age)

# Run the sims
for i in range(nsims):
    
    # Draw log_eir, serology parameters and age from prior
    log_eir = log_eir_prior.draw()
    theta = theta_prior.draw()
    age = np.random.randint(low=1, high=51)
    
    # Simulate
    x = Model.simulate(log_eir, theta, age)
    
    # Add to sim data
    xs[i,:] = x
    ps[i,:] = np.concatenate([np.atleast_1d(log_eir), theta, np.atleast_1d(age)])
    
    if ((i % 10000)==0):
        print(i/nsims*100)

0.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7000000000000001
0.8
0.8999999999999999
1.0
1.0999999999999999
1.2
1.3
1.4000000000000001
1.5
1.6
1.7000000000000002
1.7999999999999998
1.9
2.0
2.1
2.1999999999999997
2.3
2.4
2.5
2.6
2.7
2.8000000000000003
2.9000000000000004
3.0
3.1
3.2
3.3000000000000003
3.4000000000000004
3.5000000000000004
3.5999999999999996
3.6999999999999997
3.8
3.9
4.0
4.1000000000000005
4.2
4.3
4.3999999999999995
4.5
4.6
4.7
4.8
4.9
5.0
5.1
5.2
5.3
5.4
5.5
5.6000000000000005
5.7
5.800000000000001
5.8999999999999995
6.0
6.1
6.2
6.3
6.4
6.5
6.6000000000000005
6.7
6.800000000000001
6.9
7.000000000000001
7.1
7.199999999999999
7.3
7.3999999999999995
7.5
7.6
7.7
7.8
7.9
8.0
8.1
8.200000000000001
8.3
8.4
8.5
8.6
8.7
8.799999999999999
8.9
9.0
9.1
9.2
9.3
9.4
9.5
9.6
9.700000000000001
9.8
9.9
10.0
10.100000000000001
10.2
10.299999999999999
10.4
10.5
10.6
10.7
10.8
10.9
11.0
11.1
11.200000000000001
11.3
11.4
11.5
11.600000000000001
11.700000000000001
11.799999999999999
11.899999999999999
12.0

98.3
98.4
98.5
98.6
98.7
98.8
98.9
99.0
99.1
99.2
99.3
99.4
99.5
99.6
99.7
99.8
99.9


In [6]:
# Save the raw simulations
f = open('simulations/model_2/xs_raw.pkl', 'wb')
pickle.dump(xs, f)
f.close()

f = open('simulations/model_2/ps_raw.pkl', 'wb')
pickle.dump(ps, f)
f.close()

In [7]:
# Clip, log & add jitter to assist NDE
xs[xs < 1.0] = 1.0
for i in range(nsims):
    xs[i,:] = xs[i,:]+np.random.normal(loc=0.0,scale=0.01)
xs = np.log(xs)

In [8]:
# Save the simulations
f = open('simulations/model_2/xs.pkl', 'wb')
pickle.dump(xs, f)
f.close()

f = open('simulations/model_2/ps.pkl', 'wb')
pickle.dump(ps, f)
f.close()

In [ ]:
# CODE TO MANUALLY TEST SIMULATIONS AGAINST OBSERVATIONS
# Remember: simulations are for an individual at given EIR: not population average over het Exposure

# Import the trac data
tracdata = pd.read_csv('../../trac_data/summary_TRaC_data_with_covariates.csv')

# Extract the AMA titres and ages for all individuals, grouped by village
MSP = np.log(30+tracdata.MSP.values)
ages = tracdata.Age.values

# Draw parameters from prior
log_eir = log_eir_prior.draw()
theta = theta_prior.draw()

# Run sims
MSPsim = np.zeros(len(ages))
for i in range(len(ages)):
    MSPsim[i] = np.log(30+5+Model.simulate(log_eir, theta, ages[i]))

# Age range
A = np.arange(1, 51)

# Percentiles
MSP_percentiles = np.zeros((len(A), 5))
MSPsim_percentiles = np.zeros((len(A), 5))
for i in range(len(A)):
    
    # selection
    s = (ages == A[i])
    
    # Percentiles
    MSPsim_percentiles[i,:] = np.percentile(MSPsim[s], [10, 25, 50, 75, 90])# + 100
    MSP_percentiles[i,:] = np.percentile(MSP[s], [10, 25, 50, 75, 90]) #+ 100

# Plot the data and simulation percentiles as function of age
colors = ['red', 'green', 'blue', 'purple', 'yellow']
for i in range(5):
    plt.plot(A, MSP_percentiles[:,i], color = colors[i])
    plt.plot(A, MSPsim_percentiles[:,i], color = colors[i], ls = '--')
plt.show()